In [10]:
!conda install h3pandas

^C


In [12]:
import numpy as np
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point

# Set a seed for reproducibility
np.random.seed(42)

# Number of synthetic wells
n = 10

# Generate synthetic data
names = [f"Well_{i}" for i in range(1, n+1)]
num_wells = np.random.randint(1, 6, size=n)  # e.g., between 1 and 5 wells
ownership = np.random.choice([True, False], size=n)  # Inside_Prop or not
max_permit = np.round(np.random.uniform(1.0, 10.0, size=n), 2)  # in Mm3/yr
balance_areas = np.random.choice(["Area_A", "Area_B", "Area_C"], size=n)
active = [True] * n

# For Current Extraction and Value, let's assume they are the same
current_extraction = np.round(np.random.uniform(0.5, 5.0, size=n), 3)

# Costs (per m3)
opex_m3 = np.round(np.random.uniform(0.01, 0.05, size=n), 4)
drought_m3 = np.round(np.random.uniform(0.001, 0.01, size=n), 5)
co2_m3 = np.round(np.random.uniform(0.001, 0.02, size=n), 5)
env_m3 = np.round(np.random.uniform(0.001, 0.03, size=n), 5)

# Derived columns
env_cost = env_m3 * current_extraction * 1_000_000
opex = opex_m3 * current_extraction * 1_000_000
capex = np.zeros(n)  # fixed at 0

# Create random points for geometry (e.g. bounding box around Netherlands)
# Adjust lat/long range to suit your region
longs = np.random.uniform(4.0, 7.5, n)
lats = np.random.uniform(50.5, 53.5, n)
geometry = [Point(xy) for xy in zip(longs, lats)]

# Build the GeoDataFrame
active_wells_df = gpd.GeoDataFrame({
    "Name": names,
    "Num_Wells": num_wells,
    "Ownership": ownership,
    "Max_permit": max_permit,
    "Balance area": balance_areas,
    "Active": active,
    "Current Extraction": current_extraction,
    "Value": current_extraction,  # same as Current Extraction
    "OPEX_m3": opex_m3,
    "Drought_m3": drought_m3,
    "CO2_m3": co2_m3,
    "Env_m3": env_m3,
    "envCost": env_cost,
    "OPEX": opex,
    "CAPEX": capex,
    "geometry": geometry
})

print(active_wells_df)


      Name  Num_Wells  Ownership  Max_permit Balance area  Active  \
0   Well_1          4      False        2.91       Area_C    True   
1   Well_2          5      False        2.64       Area_B    True   
2   Well_3          3      False        2.65       Area_B    True   
3   Well_4          5       True        3.74       Area_C    True   
4   Well_5          5      False        5.72       Area_B    True   
5   Well_6          2       True        4.89       Area_C    True   
6   Well_7          3      False        3.62       Area_C    True   
7   Well_8          3      False        6.51       Area_A    True   
8   Well_9          3      False        2.26       Area_C    True   
9  Well_10          5      False        3.63       Area_A    True   

   Current Extraction  Value  OPEX_m3  Drought_m3   CO2_m3   Env_m3  \
0               3.234  3.234   0.0149     0.00973  0.00838  0.02340   
1               1.267  1.267   0.0298     0.00798  0.00616  0.00676   
2               0.793  0.79

In [13]:
import shapely.geometry  as sg
import json
import h3pandas as h3pd

def wells_to_h3_grid(wells_gdf, resolution=6):
    """
    Create an H3 hex grid covering all well points in 'wells_gdf'
    at the given H3 'resolution'.

    Parameters
    ----------
    wells_gdf : gpd.GeoDataFrame
        A GeoDataFrame of wells. Must be in EPSG:4326 (lat-lon).
    resolution : int
        The desired H3 resolution (0 to 15). The higher the resolution,
        the smaller the hexagons. Typical range is 5-9 for city/country scale.

    Returns
    -------
    gpd.GeoDataFrame
        A GeoDataFrame of H3 hexagon polygons that cover the bounding polygon
        around all wells, with one row per H3 cell.
    """
    # 1) Create a single polygon covering all wells
    #    - If wells are just points, you can do a convex hull or bounding box. 
    #    - Here we do a union of all points (which effectively is just a set of points)
    #      plus a small buffer so the polyfill has an area.
    #    - Alternatively, you could do bounding_poly = wells_gdf.unary_union.convex_hull
    #      if you'd like a convex hull around the wells.
    wells_union = wells_gdf.unary_union
    bounding_poly = wells_union.buffer(0.0001)  # small buffer in degrees

    # 2) Convert the bounding polygon to GeoJSON-like dict for h3.polyfill
    bounding_poly_geojson = bounding_poly.bounds
    bbox = sg.box(*bounding_poly_geojson)
    bbox_json = json.dumps(
        sg.mapping(bbox),
        indent=4
    )
    
    bbox_gdp = gpd.GeoDataFrame(bbox_json)
    bbox_h3= bbox_gdp.h3.polyfill(resolution)

    # 3) Use h3.polyfill to get all H3 cell indexes within that polygon
    # h3_indexes = h3.polygon_to_cells(bbox_json, resolution)

    # 4) Convert each H3 index to a shapely polygon
    # polygons = []
    # indexes = []
    # for h3_index in bbox_h3:
    #     # h3_to_geo_boundary returns a list of (lat, lon) in degrees
    #     boundary = h3.h3_to_geo_boundary(h3_index, geo_json=True)
    #     poly = sg.Polygon([(lon, lat) for lat, lon in boundary])
    #     polygons.append(poly)
    #     indexes.append(h3_index)

    # 5) Build a GeoDataFrame with the polygons
    # h3_gdf = gpd.GeoDataFrame({"h3_index": indexes, "geometry": polygons}, crs="EPSG:4326")
    print(bbox_h3)
    # return h3_gdf


# ---------------------------
# EXAMPLE USAGE
# ---------------------------

# Suppose you have a GeoDataFrame of wells in lat-lon or a projected CRS
# wells = gpd.read_file("path_to_wells.geojson")  # for example

# 1) (Optionally) Project wells to a local projection if needed
#    e.g., for Netherlands: wells_projected = wells.to_crs(epsg=28992)
#    Then pass wells_projected to create_hex_grid.

# 2) Create a hex grid that covers the bounding box of the wells
#    The 'radius' here is in the same units as the wells' CRS
#    If it's lat-lon degrees, 0.01 ~ about 1.1km at the equator
#    For a projected CRS (e.g. EPSG:28992), radius is in meters
hex_grid = wells_to_h3_grid(wells_gdf=active_wells_df, resolution=6)

# 3) Inspect or save the resulting grid
# print(hex_grid.head())
# hex_grid.to_file("hex_grid.geojson", driver="GeoJSON")
hex_grid.plot()


ValueError: DataFrame constructor not properly called!

In [ ]:


def add_synthetic_attributes(hex_grid):
    """
    Add random / synthetic attributes to each hex row,
    matching the structure of 'hexagons_filterd'.
    """
    n = len(hex_grid)

    # Unique IDs
    grid_ids = [f"GRID_{i+1}" for i in range(n)]

    # Balance area (categorical)
    balance_areas = np.random.choice(["Region_A", "Region_B", "Region_C"], size=n)

    # Synthetic population
    pop2022 = np.random.randint(500, 10_000, size=n)
    current_pop = np.round(pop2022*1.02, 0)  # Assume 2% growth

    # Industrial Demand
    industrial_demand = np.round(np.random.uniform(10, 100, size=n), 2)

    # For Water Demand, define some synthetic multipliers
    demand_capita = 0.15
    small_business_factor = 1.05
    water_demand = pop2022 * demand_capita * small_business_factor * 365 / 1_000_000

    # Type
    typ = np.random.choice(["Type_A", "Type_B", "Type_C"], size=n)

    # Source_Name
    source_name = np.random.choice(["River", "Well", "Lake"], size=n)

    # Now attach these columns to the hex_grid
    hex_grid["GRID_ID"] = grid_ids
    hex_grid["Balance Area"] = balance_areas
    hex_grid["Pop2022"] = pop2022
    hex_grid["Current Pop"] = current_pop
    hex_grid["Industrial Demand"] = industrial_demand
    hex_grid["Water Demand"] = water_demand
    hex_grid["Type"] = typ
    hex_grid["Source_Name"] = source_name

    return hex_grid


# 2) Add synthetic attributes
hex_grid_with_attrs = add_synthetic_attributes(hex_grid)

# Now you have a GeoDataFrame 'hex_grid_with_attrs' with polygons + random attributes
print(hex_grid_with_attrs.head())
hex_grid_with_attrs.to_file("/Assets/hex_grid_with_attrs.geojson", driver="GeoJSON")

NameError: name 'hex_grid' is not defined